In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [63]:
import py_entitymatching as em
A = em.read_csv_metadata('new1_1.csv', key='ID')
B = em.read_csv_metadata('new2_1.csv', key='ID')


In [64]:
ob = em.OverlapBlocker()
C1 = ob.block_tables(A, B, 'Name', 'Name', word_level=True, overlap_size=3, 
                    l_output_attrs=list(A.columns), 
                    r_output_attrs=list(B.columns),
                    show_progress=False)
C3 = ob.block_candset(C1, 'Resolution', 'Resolution',overlap_size=4,show_progress=False)
len(C3)

830

In [70]:
l_data = em.read_csv_metadata('labeled_data1.csv', key='_id', 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID',
                         ltable=A, rtable=B)
match_f = em.get_features_for_matching(A, B)
H = em.extract_feature_vecs(l_data, feature_table=match_f,attrs_before = None, attrs_after=['Class'])
H.fillna(value=0, inplace=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,numeric,numeric,Exact Match; Absolute Norm
1,3.5mm jack,3.5mm jack,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
2,Battery,Battery,numeric,numeric,Exact Match; Absolute Norm
3,Clock Speed,Clock Speed,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Card slot,Card slot,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
5,GPS Support,GPS Support,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
6,GPU,GPU,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Internal Storage,Internal Storage,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
8,NFC,NFC,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [71]:
train_test = em.split_train_test(H, train_proportion=0.7)
train_set = train_test['train']
test_set = train_test['test']
train_set.to_csv('training_split')
test_set.to_csv('test_split')
train_labels = train_set['Class']
test_labels = test_set['Class']
#del train_set['Class']

In [72]:
rf= em.RFMatcher()
rf.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
match_f = em.get_features_for_matching(A, B)
new = em.extract_feature_vecs(C3, feature_table=match_f,attrs_before = None)
new.fillna(value=0, inplace=True)


op = rf.predict(table=new,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'],  append=True, target_attr='Predicted_Labels')


df1=op[op['Predicted_Labels']==1]
df2=df1[['ltable_ID','rtable_ID']]

df3=op[op['Predicted_Labels']==0]
unmatched_id=df3[['ltable_ID','rtable_ID']]


The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,numeric,numeric,Exact Match; Absolute Norm
1,3.5mm jack,3.5mm jack,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
2,Battery,Battery,numeric,numeric,Exact Match; Absolute Norm
3,Clock Speed,Clock Speed,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Card slot,Card slot,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
5,GPS Support,GPS Support,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
6,GPU,GPU,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Internal Storage,Internal Storage,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
8,NFC,NFC,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


,ID,3.5mm jack,Battery,GPS Support,GPU,Internal Storage,Card slot,NFC,Name,OS,Price,Primary Camera,Clock Speed,SIM,Secondary Camera,Sensors,Weight,Resolution
1,1,Yes,5000.0,Yes,NaN,32 GB,Dedicated Slot,NaN,Moto E4 Plus,Android Nougat 7.1.1,"��9,999",13MP,1.3 GHz,Nano SIM and Nano SIM,5MP,"Fingerprint, Accelerometer, Ambient Light, Proximity Sensor",198 g,1280 x 720 Pixels


In [79]:

x = A[A['ID'].isin(df2['ltable_ID'])]
y = B[B['ID'].isin(df2['rtable_ID'])]


In [81]:
x.reset_index(drop=True)

,ID,3.5mm jack,Battery,Clock Speed,Card slot,GPS Support,GPU,Internal Storage,NFC,Name,OS,Price,Primary Camera,Resolution,SIM,Secondary Camera,Sensors,Weight
0,93,Yes,3000.0,Octa-core 2.0 GHz Cortex-A53,"microSD, up to 256 GB",Yes,Adreno 506,"64 GB, 4 GB RAM or 32 GB, 3 GB RAM",Yes,Motorola Moto G5S Plus,"Android 7.1 (Nougat), planned upgrade to Android 8.0 (Oreo)",About 210 EUR,"Dual: 13 MP, f/2.0, autofocus, dual-LED dual-tone flash, check quality","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)","Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)","8 MP, f/2.0, LED flash","Fingerprint (front-mounted), accelerometer, gyro, proximity",168 g (5.93 oz)
1,104,Yes,2800.0,Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",Yes,Mali-T720MP3,"16 GB, 2 GB RAM - Desire 72816 GB, 1.5 GB RAM - Desire 728G",NaN,HTC Desire 728 dual sim,Android 5.1.1 (Lollipop),About 240 EUR,"13 MP (f/2.2, 28mm), autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~267 ppi density)","Dual SIM (Nano-SIM, dual stand-by)","5 MP (f/2.8, 34mm), 1080p@30fps","Accelerometer, proximity, compass",153 g (5.40 oz)
2,272,Yes,3600.0,Quad-core (2x2.15 GHz Kryo & 2x1.6 GHz Kryo),"microSD, up to 256 GB (dedicated slot)",Yes,Adreno 530,"32 GB, 4 GB RAM",Yes,Samsung Galaxy S7 edge,Android 6.0 (Marshmallow),About 370 EUR,"12 MP (f/1.7, 26mm, 1/2.5"", 1.4 Âµm, Dual Pixel PDAF), phase detection autofocus, OIS, LED flash","1440 x 2560 pixels, 16:9 ratio (~534 ppi density)",Nano-SIM,"5 MP (f/1.7, 22mm, 1/4.1"", 1.34 Âµm), dual video call, Auto HDR","Fingerprint (front-mounted), accelerometer, gyro, proximity, compass, barometer, heart rate, SpO2",157 g (5.54 oz)
3,296,Yes,5000.0,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 64 GB (uses SIM 2 slot)",Yes,Mali-T720,"16 GB, 1 GB RAM",NaN,Micromax Bharat 5,Android 7.0 (Nougat),About 5700 INR,"5 MP, LED flash","720 x 1280 pixels, 16:9 ratio (~282 ppi density)","Hybrid Dual SIM (Micro-SIM/Nano-SIM, dual stand-by)","5 MP, LED flash",Accelerometer,170 g (6.00 oz)
4,361,Yes,2820.0,Quad-core 1.2 GHz Cortex-A53,"microSD, up to 32 GB",Yes,Adreno 306,"16 GB, 2 GB RAM",NaN,Micromax Canvas Play 4G Q469,Android 5.0.2 (Lollipop),NaN,"13 MP, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~267 ppi density)",Dual SIM,5 MP,"Accelerometer, proximity, compass",100 g (3.53 oz)
5,392,Yes,2000.0,Quad-core 1.3 GHz Cortex-A7,"microSD, up to 256 GB (dedicated slot)",Yes,Mali-T720,"8 GB, 1 GB RAM",No,Samsung Galaxy J2,Android 7.0 (Nougat),About 120 EUR,"5 MP, f/2.2, autofocus, LED flash","540 x 960 pixels, 16:9 ratio (~234 ppi density)","Dual SIM (Micro-SIM, dual stand-by)","2 MP, f/2.2","Accelerometer, proximity",130 g (4.59 oz)
6,462,Yes,2000.0,Quad-core 1.3 GHz Cortex-A7,"microSD, up to 32 GB",Yes,Mali-400MP2,"8 GB, 2 GB RAM",NaN,Micromax Canvas Amaze Q395,Android 5.0 (Lollipop),NaN,"13 MP, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)",5 MP,"Accelerometer, gyro, proximity, compass",-
7,492,No,3000.0,Quad-core (2x2.15 GHz Kryo & 2x1.6 GHz Kryo),"microSD, up to 256 GB (uses SIM 2 slot)",Yes,Adreno 530,"64/128 GB, 4 GB RAM",Yes,HTC U Ultra,"Android 7.0 (Nougat), upgradable to Android 8.0 (Oreo)",About 310 EUR,"12 MP (f/1.8, 26mm, 1/2.3"", 1.55 Âµm), OIS, phase detection & laser autofocus, dual-LED dual-ton...","1440 x 2560 pixels, 16:9 ratio (~513 ppi density)","Hybrid Dual SIM (Nano-SIM, dual stand-by)","16 MP, 1080p, Auto-HDR","Fingerprint (front-mounted), accelerometer, gyro, proximity, compass",170 g (6.00 oz)
8,529,Yes,5000.0,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 64 GB (uses SIM 2 slot)",Yes,Mali-T720,"16 GB, 2 GB RAM",NaN,Micromax Bharat 5 Plus,Android 7.0 (Nougat),About 7200 INR,"8 MP, LED flash","720 x 1280 pixels, 16:9 ratio (~282 ppi density)","Hybrid Dual SIM (Micro-SIM/Nano-SIM, dual stand-by)","5 MP, LED flash",Accelerometer,170 g (6.00 oz)
9,532,Yes,3600.0,Quad-core (2x2.15 GHz Kryo & 2x1.6 GHz Kryo) - G9350Octa-core (4x2.3 GHz Mongoose 

In [82]:
y.reset_index(drop=True)

,ID,3.5mm jack,Battery,GPS Support,GPU,Internal Storage,Card slot,NFC,Name,OS,Price,Primary Camera,Clock Speed,SIM,Secondary Camera,Sensors,Weight,Resolution
0,1,Yes,5000.0,Yes,NaN,32 GB,Dedicated Slot,NaN,Moto E4 Plus,Android Nougat 7.1.1,"��9,999",13MP,1.3 GHz,Nano SIM and Nano SIM,5MP,"Fingerprint, Accelerometer, Ambient Light, Proximity Sensor",198 g,1280 x 720 Pixels
1,21,Yes,3000.0,Yes,NaN,16 GB,Dedicated Slot,NaN,Samsung Galaxy J7 Nxt,Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
2,22,Yes,2500.0,Yes,Mali - T720 MP1,16 GB,Dedicated Slot,NaN,Micromax Bharat 4,Android Nougat 7,"��4,999",5MP,1.3 GHz,Micro SIM and Micro SIM,5MP,"Gravity Sensors, Proximity Sensor (By SW)",150 g,1280 x 720 Pixels
3,24,NaN,2820.0,NaN,NaN,16 GB,NaN,NaN,Micromax Canvas Play 4G Q469 Dual Sim - Moon Dust,Android Lollipop 5.0.2,"��6,499",13MP,1.2 GHz,Nano SIM,5MP,"Light Sensor, Gravity Sensor, Proximity, Magnetic Sensor",100 g,1280 x 720 Pixels
4,53,Yes,3000.0,Yes,NaN,16 GB,Dedicated Slot,NaN,Samsung Galaxy J7 Nxt,Android Nougat 7,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
5,60,NaN,4000.0,NaN,NaN,8 GB,NaN,NaN,Micromax JUICE 3,Android Lollipop V5.1,"��7,999",8MP,1.3 GHz,NaN,NaN,NaN,NaN,1280 x 720 Pixels
6,77,NaN,2000.0,NaN,NaN,8 GB,NaN,NaN,Micromax Canvas Amaze,Android Lollipop 5.1,"��6,990",13MP,1.3 GHz,NaN,NaN,NaN,NaN,1280 x 720 Pixels
7,80,Yes,2600.0,Yes,ARM Mali-T720 MP1,8 GB,Dedicated Slot,NaN,Samsung Galaxy On5,Android Marshmallow 6.0.1,"��6,990",8MP,1.3 GHz,Micro SIM,5MP,"Accelerometer, Proximity Sensor",149 g,1280 x 720 Pixels
8,81,Yes,3600.0,NaN,NaN,32 GB,NaN,Yes,Samsung Galaxy S7 Edge,Android Marshmallow 6,"��35,900",12MP,2.3 GHz,Nano SIM,5MP,"Accelerometer, Barometer, Fingerprint Sensor, Gyro Sensor, Geo-magnetic Sensor, Hall Sensor, HR ...",157 g,2560 X 1440 Pixels
9,123,NaN,1300.0,NaN,NaN,4 GB,Dedicated Slot,NaN,Micromax Bharat 2,Android Marshmallow 6,"��3,099",2MP,1.3 GHz,NaN,0.3MP,NaN,NaN,480 x 800 Pixels


In [ ]:
dfn = pd.concat([x,y], axis=1)
dfn.to_csv("matches.csv")

In [16]:
l_list1 = [93, 104, 272, 296, 361, 462, 492, 602, 705, 850, 865, 915, 920, 1014, 1084, 1106, 1136, 1184, 1326, 1392, 1474, 1511, 1561, 1625, 1775, 1926, 1932, 2038, 2072, 2108, 2124, 2139, 2150, 2167, 2234, 2241, 2292, 2333, 2390, 2410, 2470, 2534, 2591, 2594, 2600, 2666, 2680, 2695, 2710, 2754, 2831, 2885, 3025, 3127, 3175, 3224]

In [17]:
l_list2 = list(unmatched_id['ltable_ID'])
rlist = list(unmatched_id['rtable_ID'])
set_l = set(l_list1) | set(l_list2)
set_r = set(rlist)

final_l_list = list(set_l)
final_rlist = list(set_r)

In [18]:
A.head()

,ID,3.5mm jack,Battery,Clock Speed,Card slot,GPS Support,GPU,Internal Storage,NFC,Name,OS,Price,Primary Camera,Resolution,SIM,Secondary Camera,Sensors,Weight
0,0,Yes,1900.0,Quad-core 1.2 GHz Cortex-A53,"microSD, up to 64 GB (uses SIM 2 slot)",Yes,Adreno 306,"16 GB, 1 GB RAM",Yes,Samsung Galaxy A3 Duos,"Android 4.4.4 (KitKat), upgradable to 5.0.2 (Lollipop)",About 230 EUR,"8 MP (f/2.4, 31mm), autofocus, LED flash","540 x 960 pixels, 16:9 ratio (~245 ppi density)","Hybrid Dual SIM (Nano-SIM, dual stand-by)","5 MP (f/2.2, 23mm)","Accelerometer, proximity, compass",110.3 g (3.88 oz)
1,1,Yes,950.0,NaN,"microSD, up to 32 GB (dedicated slot)",No,NaN,4 MB RAM,NaN,Nokia 108 Dual SIM,NaN,About 30 EUR,VGA,128 x 160 pixels (~114 ppi density),"Dual SIM (Mini-SIM, dual stand-by)",No,NaN,70.2 g (2.47 oz)
2,2,No,NaN,NaN,No,No,NaN,NaN,NaN,Samsung M310,NaN,About 80 EUR,VGA,128 x 160 pixels (~121 ppi density),Mini-SIM,No,NaN,87 g (3.07 oz)
3,3,Yes,2200.0,Quad-core 1.2 GHz Cortex-A7,"microSD, up to 32 GB",Yes,Adreno 302,"8 GB ROM, 1 GB RAM",NaN,BLU Win HD,Microsoft Windows Phone 8.1,About 150 EUR,"8 MP, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Dual SIM (Micro-SIM),2 MP,"Accelerometer, proximity",139 g (4.90 oz)
4,4,No,800.0,NaN,"microSD, up to 8 GB (dedicated slot)",No,NaN,NaN,NaN,Celkon C369,NaN,About 30 EUR,"1.3 MP, LED flash",176 x 220 pixels (~128 ppi density),Dual SIM (Mini-SIM),No,NaN,102 g (3.60 oz)


In [19]:
A[A['ID'] == 93]

,ID,3.5mm jack,Battery,Clock Speed,Card slot,GPS Support,GPU,Internal Storage,NFC,Name,OS,Price,Primary Camera,Resolution,SIM,Secondary Camera,Sensors,Weight
93,93,Yes,3000.0,Octa-core 2.0 GHz Cortex-A53,"microSD, up to 256 GB",Yes,Adreno 506,"64 GB, 4 GB RAM or 32 GB, 3 GB RAM",Yes,Motorola Moto G5S Plus,"Android 7.1 (Nougat), planned upgrade to Android 8.0 (Oreo)",About 210 EUR,"Dual: 13 MP, f/2.0, autofocus, dual-LED dual-tone flash, check quality","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)","Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)","8 MP, f/2.0, LED flash","Fingerprint (front-mounted), accelerometer, gyro, proximity",168 g (5.93 oz)


In [20]:
C3[C3['ltable_ID']==93]['ltable_Name']

213    Motorola Moto G5S Plus
441    Motorola Moto G5S Plus
Name: ltable_Name, dtype: object

In [21]:
filtered_l = A[A['ID'].isin(final_l_list)]
filtered_r = B[B['ID'].isin(final_rlist)]
df = filtered_l.append(filtered_r)
df


,3.5mm jack,Battery,Card slot,Clock Speed,GPS Support,GPU,ID,Internal Storage,NFC,Name,OS,Price,Primary Camera,Resolution,SIM,Secondary Camera,Sensors,Weight
8,Yes,3000.0,"microSD, up to 32 GB (dedicated slot)",Quad-core 1.3 GHz Cortex-A7,Yes,Mali-400MP2,8,"8 GB, 2 GB RAM",NaN,Micromax Canvas Juice 2 AQ5001,Android 5.0 (Lollipop),About 130 EUR,"8 MP, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)",2 MP,"Accelerometer, proximity",170 g (6.00 oz)
93,Yes,3000.0,"microSD, up to 256 GB",Octa-core 2.0 GHz Cortex-A53,Yes,Adreno 506,93,"64 GB, 4 GB RAM or 32 GB, 3 GB RAM",Yes,Motorola Moto G5S Plus,"Android 7.1 (Nougat), planned upgrade to Android 8.0 (Oreo)",About 210 EUR,"Dual: 13 MP, f/2.0, autofocus, dual-LED dual-tone flash, check quality","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)","Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)","8 MP, f/2.0, LED flash","Fingerprint (front-mounted), accelerometer, gyro, proximity",168 g (5.93 oz)
104,Yes,2800.0,"microSD, up to 256 GB (dedicated slot)",Octa-core 1.3 GHz Cortex-A53,Yes,Mali-T720MP3,104,"16 GB, 2 GB RAM - Desire 72816 GB, 1.5 GB RAM - Desire 728G",NaN,HTC Desire 728 dual sim,Android 5.1.1 (Lollipop),About 240 EUR,"13 MP (f/2.2, 28mm), autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~267 ppi density)","Dual SIM (Nano-SIM, dual stand-by)","5 MP (f/2.8, 34mm), 1080p@30fps","Accelerometer, proximity, compass",153 g (5.40 oz)
122,Yes,3050.0,"microSD, up to 64 GB (dedicated slot)",Quad-core 1.3 GHz Cortex-A53,Yes,Mali-T720,122,"16 GB, 3 GB RAM",NaN,Micromax Canvas 2 Q4310,Android 7.0 (Nougat),About 12000 INR,"13 MP, f/2.0, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)","5 MP, f/2.0","Fingerprint (front-mounted), accelerometer, proximity",160 g (5.64 oz)
171,Yes,2100.0,"microSD, up to 32 GB (dedicated slot)",Octa-core 1.7 GHz Cortex-A7,Yes,Mali-450MP4,171,"8 GB, 1 GB RAM",NaN,HTC Desire 620G dual sim,Android 4.4.4 (KitKat),About 190 EUR,"8 MP, f/2.4, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)","5 MP, 1080p","Accelerometer, proximity",160 g (5.64 oz)
236,Yes,2000.0,"microSD, up to 32 GB (dedicated slot)",Octa-core 1.4 GHz Cortex-A7,Yes,Mali-450MP4,236,"4 GB, 1 GB RAM",NaN,HTC Desire 616 dual sim,Android 4.2.2 (Jelly Bean),About 180 EUR,"8 MP, autofocus, LED flash, check quality","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Dual SIM (Mini-SIM/ Micro-SIM),"2 MP, 720p","Accelerometer, proximity",150 g (5.29 oz)
251,Yes,3600.0,"microSD, up to 256 GB (dedicated slot)",Octa-core 1.9 GHz Cortex-A53,Yes,Mali-T830MP3,251,"32 GB, 3 GB RAM",Yes,Samsung Galaxy A7,"Android 6.0.1 (Marshmallow), upgradable to 7.0 (Nougat)",About 390 EUR,"16 MP (f/1.9, 27mm), autofocus, LED flash, check quality","1080 x 1920 pixels, 16:9 ratio (~386 ppi density)","Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)","16 MP, f/1.9, 1080p","Fingerprint (front-mounted), accelerometer, gyro, proximity, compass, barometer",186 g (6.56 oz)
272,Yes,3600.0,"microSD, up to 256 GB (dedicated slot)",Quad-core (2x2.15 GHz Kryo & 2x1.6 GHz Kryo),Yes,Adreno 530,272,"32 GB, 4 GB RAM",Yes,Samsung Galaxy S7 edge,Android 6.0 (Marshmallow),About 370 EUR,"12 MP (f/1.7, 26mm, 1/2.5"", 1.4 Âµm, Dual Pixel PDAF), phase detection autofocus, OIS, LED flash","1440 x 2560 pixels, 16:9 ratio (~534 ppi density)",Nano-SIM,"5 MP (f/1.7, 22mm, 1/4.1"", 1.34 Âµm), dual video call, Auto HDR","Fingerprint (front-mounted), accelerometer, gyro, proximity, compass, barometer, heart rate, SpO2",157 g (5.54 oz)
296,Yes,5000.0,"microSD, up to 64 GB (uses SIM 2 slot)",Quad-core 1.3 GHz Cortex-A53,Yes,Mali-T720,296,"16 GB, 1 GB RAM",NaN,Micromax Bharat 5,Android 7.0 (Nougat),About 5700 INR,"5 MP, LED flash","720 x 1280 pixels, 16:9 ratio (~282 ppi density)","Hybrid Dual SIM (Micro-SIM/Nano-SIM, dual stand-by)","5 MP, LED flash",Acce

In [22]:
len(final_l_list)

144

In [23]:
len(final_rlist)

137

In [24]:
rlist2 = [1, 21, 22, 24, 60, 77, 80, 81, 123, 133, 134, 162, 165, 171, 179, 231, 239, 243, 343, 349, 355, 358, 486, 504, 548, 568, 629, 636, 665, 749, 755, 830, 861, 921, 1006, 1010, 1066, 1077, 1078, 1158, 1163, 1398, 1404, 1407, 1418, 1473, 1488, 1493]

In [25]:
set_r2 = set(rlist2)

In [26]:
set_r_inter = set_r & set_r2

In [27]:
final_rlist = list(set_r - set_r_inter)

In [28]:
filtered_l = A[A['ID'].isin(final_l_list)]
filtered_r = B[B['ID'].isin(final_rlist)]
df = filtered_l.append(filtered_r)
df

,3.5mm jack,Battery,Card slot,Clock Speed,GPS Support,GPU,ID,Internal Storage,NFC,Name,OS,Price,Primary Camera,Resolution,SIM,Secondary Camera,Sensors,Weight
8,Yes,3000.0,"microSD, up to 32 GB (dedicated slot)",Quad-core 1.3 GHz Cortex-A7,Yes,Mali-400MP2,8,"8 GB, 2 GB RAM",NaN,Micromax Canvas Juice 2 AQ5001,Android 5.0 (Lollipop),About 130 EUR,"8 MP, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)",2 MP,"Accelerometer, proximity",170 g (6.00 oz)
93,Yes,3000.0,"microSD, up to 256 GB",Octa-core 2.0 GHz Cortex-A53,Yes,Adreno 506,93,"64 GB, 4 GB RAM or 32 GB, 3 GB RAM",Yes,Motorola Moto G5S Plus,"Android 7.1 (Nougat), planned upgrade to Android 8.0 (Oreo)",About 210 EUR,"Dual: 13 MP, f/2.0, autofocus, dual-LED dual-tone flash, check quality","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)","Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)","8 MP, f/2.0, LED flash","Fingerprint (front-mounted), accelerometer, gyro, proximity",168 g (5.93 oz)
104,Yes,2800.0,"microSD, up to 256 GB (dedicated slot)",Octa-core 1.3 GHz Cortex-A53,Yes,Mali-T720MP3,104,"16 GB, 2 GB RAM - Desire 72816 GB, 1.5 GB RAM - Desire 728G",NaN,HTC Desire 728 dual sim,Android 5.1.1 (Lollipop),About 240 EUR,"13 MP (f/2.2, 28mm), autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~267 ppi density)","Dual SIM (Nano-SIM, dual stand-by)","5 MP (f/2.8, 34mm), 1080p@30fps","Accelerometer, proximity, compass",153 g (5.40 oz)
122,Yes,3050.0,"microSD, up to 64 GB (dedicated slot)",Quad-core 1.3 GHz Cortex-A53,Yes,Mali-T720,122,"16 GB, 3 GB RAM",NaN,Micromax Canvas 2 Q4310,Android 7.0 (Nougat),About 12000 INR,"13 MP, f/2.0, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)","5 MP, f/2.0","Fingerprint (front-mounted), accelerometer, proximity",160 g (5.64 oz)
171,Yes,2100.0,"microSD, up to 32 GB (dedicated slot)",Octa-core 1.7 GHz Cortex-A7,Yes,Mali-450MP4,171,"8 GB, 1 GB RAM",NaN,HTC Desire 620G dual sim,Android 4.4.4 (KitKat),About 190 EUR,"8 MP, f/2.4, autofocus, LED flash","720 x 1280 pixels, 16:9 ratio (~294 ppi density)","Dual SIM (Micro-SIM, dual stand-by)","5 MP, 1080p","Accelerometer, proximity",160 g (5.64 oz)
236,Yes,2000.0,"microSD, up to 32 GB (dedicated slot)",Octa-core 1.4 GHz Cortex-A7,Yes,Mali-450MP4,236,"4 GB, 1 GB RAM",NaN,HTC Desire 616 dual sim,Android 4.2.2 (Jelly Bean),About 180 EUR,"8 MP, autofocus, LED flash, check quality","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Dual SIM (Mini-SIM/ Micro-SIM),"2 MP, 720p","Accelerometer, proximity",150 g (5.29 oz)
251,Yes,3600.0,"microSD, up to 256 GB (dedicated slot)",Octa-core 1.9 GHz Cortex-A53,Yes,Mali-T830MP3,251,"32 GB, 3 GB RAM",Yes,Samsung Galaxy A7,"Android 6.0.1 (Marshmallow), upgradable to 7.0 (Nougat)",About 390 EUR,"16 MP (f/1.9, 27mm), autofocus, LED flash, check quality","1080 x 1920 pixels, 16:9 ratio (~386 ppi density)","Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)","16 MP, f/1.9, 1080p","Fingerprint (front-mounted), accelerometer, gyro, proximity, compass, barometer",186 g (6.56 oz)
272,Yes,3600.0,"microSD, up to 256 GB (dedicated slot)",Quad-core (2x2.15 GHz Kryo & 2x1.6 GHz Kryo),Yes,Adreno 530,272,"32 GB, 4 GB RAM",Yes,Samsung Galaxy S7 edge,Android 6.0 (Marshmallow),About 370 EUR,"12 MP (f/1.7, 26mm, 1/2.5"", 1.4 Âµm, Dual Pixel PDAF), phase detection autofocus, OIS, LED flash","1440 x 2560 pixels, 16:9 ratio (~534 ppi density)",Nano-SIM,"5 MP (f/1.7, 22mm, 1/4.1"", 1.34 Âµm), dual video call, Auto HDR","Fingerprint (front-mounted), accelerometer, gyro, proximity, compass, barometer, heart rate, SpO2",157 g (5.54 oz)
296,Yes,5000.0,"microSD, up to 64 GB (uses SIM 2 slot)",Quad-core 1.3 GHz Cortex-A53,Yes,Mali-T720,296,"16 GB, 1 GB RAM",NaN,Micromax Bharat 5,Android 7.0 (Nougat),About 5700 INR,"5 MP, LED flash","720 x 1280 pixels, 16:9 ratio (~282 ppi density)","Hybrid Dual SIM (Micro-SIM/Nano-SIM, dual stand-by)","5 MP, LED flash",Acce

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
data=pd.read_csv("Merged2.csv")
sec_cam=data['Secondary Camera'].apply(lambda x:str(x).split("MP")[0])
data.insert(1,"sec_cam",sec_cam)





In [43]:
import py_entitymatching as em
A = em.read_csv_metadata('new1_1.csv', key='ID')#Files before cleaning
B = em.read_csv_metadata('new2_1.csv', key='ID')


rf= em.RFMatcher()
rf.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
match_f = em.get_features_for_matching(A, B)
new = em.extract_feature_vecs(C3, feature_table=match_f,attrs_before = None)
new.fillna(value=0, inplace=True)


op = rf.predict(table=new,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'],  append=True, target_attr='Predicted_Labels')


df1=op[op['Predicted_Labels']==1]
df1.to_csv("matches.csv")
df2=df1[['ltable_ID','rtable_ID']]
df2.to_csv("matching.csv")
df3=op[op['Predicted_Labels']==0]

unmatched_id=df3[['ltable_ID','rtable_ID']]


l_lmatched = set([93, 104, 272, 296, 361, 462, 492, 602, 705, 850, 865, 915, 920, 1014, 1084, 1106, 1136, 1184, 1326, 1392, 1474, 1511, 1561, 1625, 1775, 1926, 1932, 2038, 2072, 2108, 2124, 2139, 2150, 2167, 2234, 2241, 2292, 2333, 2390, 2410, 2470, 2534, 2591, 2594, 2600, 2666, 2680, 2695, 2710, 2754, 2831, 2885, 3025, 3127, 3175, 3224])
r_matched = set([93, 104, 272, 296, 361, 462, 492, 602, 705, 850, 865, 915, 920, 1014, 1084, 1106, 1136, 1184, 1326, 1392, 1474, 1511, 1561, 1625, 1775, 1926, 1932, 2038, 2072, 2108, 2124, 2139, 2150, 2167, 2234, 2241, 2292, 2333, 2390, 2410, 2470, 2534, 2591, 2594, 2600, 2666, 2680, 2695, 2710, 2754, 2831, 2885, 3025, 3127, 3175, 3224])
l_unmatched = set(list(unmatched_id['ltable_ID']))
r_unmatched = set(list(unmatched_id['rtable_ID']))
op1= l_lmatched.union(l_unmatched)
op2= r_unmatched.difference(r_matched) 
out1 = A[A['ID'].isin(op1)]
out2 = B[B['ID'].isin(op2)]
datanew = out1.append(out2)
datanew.to_csv("final_list.csv")

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,numeric,numeric,Exact Match; Absolute Norm
1,3.5mm jack,3.5mm jack,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
2,Battery,Battery,numeric,numeric,Exact Match; Absolute Norm
3,Clock Speed,Clock Speed,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Card slot,Card slot,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
5,GPS Support,GPS Support,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
6,GPU,GPU,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Internal Storage,Internal Storage,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
8,NFC,NFC,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Name,Name,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09
